In [ ]:
!pip install gdown

In [ ]:
import gdown
URL='https://drive.google.com/uc?id=__enter the ID here__'
output='plant-diseases-augmented.zip'
gdown.download(URL, output, quiet=False)

In [ ]:
!unzip -q /kaggle/working/plant-diseases-augmented.zip -d /kaggle/working/

In [ ]:
import os
os.remove('/kaggle/working/plant-diseases-augmented.zip')

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install tensorflow-gpu==2.6.4

In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np

In [ ]:
train_dir= '/kaggle/working/plant disease/train'
test_dir= '/kaggle/working/plant disease/test'
val_dir= '/kaggle/working/plant disease/val'

In [ ]:
# Resizing all images using tf.keras.preprocessing.image_dataset_from_directory 
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(224, 224),
                                                                 label_mode='categorical', batch_size=32,
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(224, 224),
                                                                 label_mode='categorical', batch_size=32,
                                                                )

In [ ]:
# Instantiate base_model with pre-trained weights
base_model= tf.keras.applications.efficientnet.EfficientNetB0(weights= 'imagenet', input_shape=(224, 224, 3), include_top=False) 

In [ ]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [ ]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (224, 224, 3), name="input_layer")

In [ ]:
x= base_model(inputs, training= False)

In [ ]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)
print(f"Shape after passing GlobalAveragePooling layer is {x.shape}")

In [ ]:
# Output layer
outputs = tf.keras.layers.Dense(38, activation="softmax", name="output_layer")(x)

In [ ]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [ ]:
model.summary()

In [ ]:
# Compile the model
model.compile(loss= tf.keras.losses.CategoricalCrossentropy(), 
              optimizer = tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

**TRAINING**

In [ ]:
# Fit the model
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=40, validation_data=val_data, validation_steps=len(val_data))

In [ ]:
# Saving model (.h5 extension)
model.save('plant_diseases_model.h5')

**FINE-TUNE MODEL**

In [ ]:
'''Unfreeze the base_model. Note that it keeps running in inference mode
   since we passed `training=False` when calling it. This means that
   the batchnormalization layers won't update their batch statistics.
   This prevents the batchnormalization layers from undoing all the training, 
   we've done so far.'''

base_model.trainable= True

In [ ]:
#from keras import load_model
loaded_model = tf.keras.models.load_model('plant_diseases_model.h5')
print('Loaded model from kaggle working directory')

In [ ]:
loaded_model.summary()

In [ ]:
# Compile the model
loaded_model.compile(loss= tf.keras.losses.CategoricalCrossentropy(), 
              optimizer = tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])

In [ ]:
loaded_model.fit(train_data, epochs=10, validation_data=val_data)

In [ ]:
# Saving fine-tune model (.h5 extension)
model.save('fine_tune_plant_diseases_model.h5')

**PREDICTION**

In [ ]:
fine_tune_model = tf.keras.models.load_model('fine_tune_plant_diseases_model.h5')
print('Loaded fine-tune model from kaggle working directory')

label= ['Blueberry___healthy',
        'Peach___Bacterial_spot',
        'Tomato___Leaf_Mold',
        'Grape___Black_rot',
        'Corn_(maize)___Common_rust_',
        'Strawberry___healthy',
        'Apple___healthy',
        'Tomato___Target_Spot',
        'Tomato___Bacterial_spot',
        'Tomato___Spider_mites Two-spotted_spider_mite',
        'Cherry_(including_sour)___healthy',
        'Apple___Cedar_apple_rust',
        'Cherry_(including_sour)___Powdery_mildew',
        'Peach___healthy',
        'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
        'Raspberry___healthy',
        'Squash___Powdery_mildew',
        'Tomato___Tomato_mosaic_virus',
        'Grape___healthy',
        'Strawberry___Leaf_scorch',
        'Tomato___Late_blight',
        'Potato___Late_blight',
        'Tomato___Early_blight',
        'Potato___healthy',
        'Tomato___Septoria_leaf_spot',
        'Orange___Haunglongbing_(Citrus_greening)',
        'Pepper,_bell___healthy',
        'Corn_(maize)___Northern_Leaf_Blight',
        'Potato___Early_blight',
        'Tomato___healthy',
        'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
        'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
        'Grape___Esca_(Black_Measles)',
        'Corn_(maize)___healthy',
        'Apple___Apple_scab',
        'Apple___Black_rot',
        'Soybean___healthy',
        'Pepper,_bell___Bacterial_spot']
path= './plant disease/test/Tomato___healthy/d6587652-47aa-4518-9534-468194f22953___RS_HL 0595.JPG'
test_image = image.load_img(path, target_size=(224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = loaded_model.predict(test_image)
fresult = np.max(result)
label = label[result.argmax()]
print(label)